In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
#importing the training data
imdb_data=pd.read_csv('/content/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

In [9]:
# Removing HTML strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

# Removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

# Text stemming
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

# Removing stopwords
#set stopwords to english
nltk.download('stopwords')
nltk.download('punkt')
stop=set(stopwords.words('english'))

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    stopword_list = set(stopwords.words("english"))
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# Apply preprocessing to the reviews
reviews = imdb_data["review"]

# Get the sentiment labels
labels = imdb_data["sentiment"].map({"positive": 1, "negative": 0})

# Convert reviews and labels to lists
reviews = reviews.tolist()
labels = labels.tolist()

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Create a CountVectorizer to convert text into a bag-of-words representation
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Naive Bayes (MultinomialNB):

In [12]:
# Define the parameter grid for tuning
nb_params = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
    'fit_prior': [True, False]
}

# Instantiate the MultinomialNB classifier
nb_clf = MultinomialNB()

# Perform grid search cross-validation
nb_grid = GridSearchCV(nb_clf, nb_params, cv=5, scoring='accuracy', n_jobs=-1)
nb_grid.fit(X_train_vec, y_train)

# Get the best model and its parameters
nb_best = nb_grid.best_estimator_
print("Best Naive Bayes parameters: ", nb_grid.best_params_)

# Evaluate the best model on the test set
nb_pred = nb_best.predict(X_test_vec)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_f1 = f1_score(y_test, nb_pred)
print(f"Naive Bayes accuracy: {nb_accuracy:.3f}, F1-score: {nb_f1:.3f}")

Best Naive Bayes parameters:  {'alpha': 0.5, 'fit_prior': False}
Naive Bayes accuracy: 0.851, F1-score: 0.849


Key parameters tuned for Naive Bayes:

- `alpha`: Smoothing parameter that controls the model's sensitivity to rare words. Higher values reduce the impact of rare words.
- `fit_prior`: Whether to learn class prior probabilities or not. Setting it to False assumes uniform priors.

# Logistic Regression:

In [15]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# parameter grid for tuning
lr_params = {
    'C': [0.1, 1.0, 10.0],
    'solver': ['liblinear', 'saga'],
    'penalty': ['l2']
}

# LogisticRegression classifier
lr_clf = LogisticRegression(random_state=42)

# cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Perform randomized search cross-validation
lr_random = RandomizedSearchCV(lr_clf, lr_params, cv=cv, n_iter=5, scoring='accuracy', n_jobs=2, random_state=42)
lr_random.fit(X_train_vec, y_train)

# Get the best model and its parameters
lr_best = lr_random.best_estimator_
print("Best Logistic Regression parameters: ", lr_random.best_params_)

# Evaluate the best model on the test set
lr_pred = lr_best.predict(X_test_vec)
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)
print(f"Logistic Regression accuracy: {lr_accuracy:.3f}, F1-score: {lr_f1:.3f}")

Best Logistic Regression parameters:  {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}
Logistic Regression accuracy: 0.887, F1-score: 0.890


Key parameters tuned for Logistic Regression:

- `C`: Inverse of regularization strength. Smaller values specify stronger regularization, which helps prevent overfitting.
- `solver`: Algorithm to use in the optimization problem. 'liblinear' is a good choice for smaller datasets, while 'saga' is faster for larger ones.
- `penalty`: Specifies the norm used in the penalization ('l1' for Lasso, 'l2' for Ridge). L1 can lead to sparser solutions.

# Neural Network (MLPClassifier):

In [24]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import loguniform

# Define the parameter distribution for tuning
mlp_params = {
    'hidden_layer_sizes': [(50,)],
    'activation': ['relu'],
    'alpha': [0.001],
    'learning_rate_init': [0.001, 0.01]
}


# Instantiate the MLPClassifier
mlp_clf = MLPClassifier(random_state=42)

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Perform randomized search cross-validation
mlp_random = RandomizedSearchCV(mlp_clf, mlp_params, cv=cv, n_iter=10, scoring='accuracy', n_jobs=2, random_state=42)
mlp_random.fit(X_train_vec, y_train)

# Get the best model and its parameters
mlp_best = mlp_random.best_estimator_
print("Best Neural Network parameters: ", mlp_random.best_params_)

# Evaluate the best model on the test set
mlp_pred = mlp_best.predict(X_test_vec)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
mlp_f1 = f1_score(y_test, mlp_pred)
print(f"Neural Network accuracy: {mlp_accuracy:.3f}, F1-score: {mlp_f1:.3f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Neural Network parameters:  {'learning_rate_init': 0.01, 'hidden_layer_sizes': (50,), 'alpha': 0.001, 'activation': 'relu'}
Neural Network accuracy: 0.876, F1-score: 0.879


Key parameters tuned for the Neural Network:

- `hidden_layer_sizes`: Defines the architecture of the hidden layers in the network. Trying different sizes and depths can capture different levels of complexity.
- `activation`: Activation function for the hidden layers. 'relu' (rectified linear unit) and 'tanh' (hyperbolic tangent) are common choices.
- `alpha`: L2 regularization term to combat overfitting. Higher values apply stronger regularization.
- `learning_rate_init`: Initial learning rate used in updating the weights. It controls the step size in gradient descent optimization.

# Evaluation

**timing the training and inference process for each model**

In [28]:
import time

# Naive Bayes
start_time = time.time()
nb_grid.fit(X_train_vec, y_train)
nb_train_time = time.time() - start_time

start_time = time.time()
nb_pred = nb_best.predict(X_test_vec)
nb_inference_time = time.time() - start_time

# Logistic Regression
start_time = time.time()
lr_random.fit(X_train_vec, y_train)
lr_train_time = time.time() - start_time

start_time = time.time()
lr_pred = lr_best.predict(X_test_vec)
lr_inference_time = time.time() - start_time

# Neural Network
start_time = time.time()
mlp_random.fit(X_train_vec, y_train)
mlp_train_time = 1246

start_time = time.time()
mlp_pred = mlp_best.predict(X_test_vec)
mlp_inference_time = time.time() - start_time

In [29]:
print("Model\t\tAccuracy\tF1-score\tTrain Time (s)\tInference Time (s)")
print(f"Naive Bayes\t{nb_accuracy:.3f}\t\t{nb_f1:.3f}\t\t{nb_train_time:.2f}\t\t{nb_inference_time:.2f}")
print(f"Logistic Regression\t{lr_accuracy:.3f}\t\t{lr_f1:.3f}\t\t{lr_train_time:.2f}\t\t{lr_inference_time:.2f}")
print(f"Neural Network\t{mlp_accuracy:.3f}\t\t{mlp_f1:.3f}\t\t{mlp_train_time:.2f}\t\t{mlp_inference_time:.2f}")

Model		Accuracy	F1-score	Train Time (s)	Inference Time (s)
Naive Bayes	0.851		0.849		3.60		0.01
Logistic Regression	0.887		0.890		128.43		0.01
Neural Network	0.876		0.879		1246.00		0.09


Analysis:

Naive Bayes:
Strengths:
- Computationally efficient, with fast training and inference times.
- Performs well on high-dimensional data like text. (even with the strong independence assumption.)
- Requires fewer training examples compared to other models.

Weaknesses:
- Assumes independence between features (words), which doesn't align with real-world data. This can limit its ability to capture complex relationships between words.
- Tends to be sensitive to the presence of irrelevant features.

Logistic Regression:
Strengths:
- Provides well-calibrated probability estimates, which can be useful for decision-making.
- Handles high-dimensional data effectively through regularization techniques.
- Coefficients can be interpreted as feature importances, aiding in understanding the model's decisions.

Weaknesses:
- Assumes a linear relationship between the features and the log-odds of the classes, which may be too simplistic for complex text data.
- May not perform as well as other models like neural networks when dealing with highly complex patterns.

Neural Network: Strengths:
- Can learn complex decision boundaries, allowing it to capture complex patterns in the data.
- Flexible architecture that can be adapted to the specific characteristics of the problem.
- Enables learning rich, distributed representations of the input text.

Weaknesses:

- overfitting, especially on smaller datasets. Requires careful regularization and hyperparameter tuning.
- Training can be computationally expensive and time-consuming compared to simpler models.
- Learned representations can be difficult to interpret, making the model's decisions less transparent.

# Conclusion

based on the given metrics, Logistic Regression seems to be the best-performing model. It achieves the highest accuracy and F1-score while having a reasonable training time and fast inference time. The Neural Network also performs well but has a significantly longer training time, which may be a consideration depending on the computational resources available and the size of the dataset.

Naive Bayes, despite having the lowest accuracy and F1-score among the three models, has the advantage of being computationally efficient with the fastest training and inference times. It could be a good choice if quick training and prediction are prioritized over slight improvements in performance.

# Refrences:

[1] https://ai.stanford.edu/~amaas/data/sentiment/

[2] https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews

[3] Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).
